# Enron Dataset Preprocessing : Format the dataset into time series

In this notebook, we format the parsed dataset into a multivariate time series.

## Step 1: Import libraries

In [ ]:
import os
import re
import sys
import time
from datetime import datetime
from collections import defaultdict
from collections import Counter

import pandas as pd

## Step 2: Load the parsed dataset

The parsed dataset should first be generated from the previous notebook.

In [ ]:
df = pd.read_csv('enron_dataset_raw.csv', encoding='utf-8', index_col=0)
df.shape # Shape: (517401, 13)

In [ ]:
df[['user', 'From', 'To', 'X-Origin', 'X-From', 'X-To', 'date', 'timestamp', 'file']].head(3)

## Step 2: Preprocess the dataframe

### Step 2.1: Basic cleaning & filtering

Deal with `Na` values. I.e. Drop columns with mostly `Na` and drop rows with remaining `Na`.

In [ ]:
# Drop columns Cc, Bcc, X-cc, X-bcc which are mostly Na
df = df.drop(columns=['Cc', 'Bcc', 'X-cc', 'X-bcc'])

# Remove rows with missing values is empty strings
df = df.dropna()

Check the shape.

In [ ]:
df.shape # Shape: (495547, 9)

Apply some basic manual fixes to some fields.

In [ ]:
# Rename columns
df = df.rename(columns={'user': 'user_id'})

# Get the folder of the email from the `file` field
df['folder'] = df.file.apply(lambda fpath: fpath.split('/')[-2])

# Remove doublon from a user that appears to be duplicated (i.e. has a single email address for two different usernames)
df = df[df['user_id'] != 'whalley-l']

# Remove 'Taylor, Michael E' mixed up with 'Mark Taylor' in user 'taylor-m' that appears to be duplicated (i.e. has a single email address for two different usernames)
df = df[~df['X-From'].apply(lambda s: 'Taylor, Michael E' in s)]

Filter mails from/to addresses outside of enron.

In [ ]:
mask_from_enron = df['From'].apply(lambda s: '@enron.com' in s)
mask_to_enron = df['To'].apply(lambda s: '@enron.com' in s)
df = df[mask_to_enron & mask_from_enron]

Filter users with not enough mails sent.

In [ ]:
# Minimum number of emails per use to pass filter
MIN_MAIL_COUNT = 50

df_user_count = df['user_id'].value_counts() 
users_to_keep = set(df_user_count[df_user_count > MIN_MAIL_COUNT].index)
mask_has_enough_mails = df['user_id'].apply(lambda u: u in users_to_keep)
df = df[mask_has_enough_mails]

Generate unique index for each email.

In [ ]:
# Reset the index
df = df.reset_index(drop=True)

# Generate a unique `id` for each email before we duplicated them  by splitting the `To` fields
df['mail_id'] = df.index.copy()

Visualize a few examples.

In [ ]:
print(df.shape) # Shape: (347051, 11)
df.head(3) 

### Step 2.2: Identify senders in inbox folders

#### Match user ids with X-From fields

Some emails occur in the inbox of some users but the sent emails of their senders. To augment the dataset with these emails, we match the users with their `X-From` field.

In [ ]:
from collections import Counter

def clean_xname_str(s):
    return [re.sub(' <.+>', '', re.sub('\n\t', '', addr)) for addr in s.split(', ')]

def get_user_sent_mask(user):
    return (df.user_id == user) & pd.concat([(df.folder == folder) for folder in ['sent', 'sent_items', '_sent_mail']], axis=1).any(axis=1)

def clean_match(match):
    match_dict = match.groupdict()
    if match_dict['email']:
        return match_dict['email'].strip("'")
    elif match_dict['name']:
        return re.sub(' <.+>', '', match_dict['name']).strip().lower()
    elif match_dict['single_name']:
        return match_dict['single_name'].strip().lower()
    else:
        raise ValueError('Invalid match')
    
re_is_xname_str = r"(?P<name>[\w. ]+, [\w. ]+( \([\w. ]+\))? <[^>]+>)"
re_is_mail_str = r"(?P<email>'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')"
re_is_single_name_str = r"^(?P<single_name>[\w. ]+(, [\w. ]+)?)$"
reg_xname = re.compile('|'.join([re_is_xname_str, re_is_mail_str, re_is_single_name_str]))

In [ ]:
xfrom_user_count_dict = {}
for user in df.user_id.unique():
    user_sent_mask = get_user_sent_mask(user)
    if sum(user_sent_mask) > 0:
        arr = np.hstack([list(map(clean_match, reg_xname.finditer(xfrom_str))) for xfrom_str in df[user_sent_mask]['X-From'].values])
        xfrom_user_count_dict[user] = sorted(Counter(arr).items(), key=lambda e: e[1], reverse=True)
    else:
        print('Error on user', user)

In [ ]:
filtered_xfrom_user_dict = {}

# Filter names that does not contain the lastname of the user
for user, name_list in xfrom_user_count_dict.items():
    if len(name_list) > 1:
        cleaned_name_list = []
        for name,_ in name_list:
            lastname = user.split('-')[0]
            if lastname in name:
                cleaned_name_list.append(name)
    else:
        cleaned_name_list = [name_list[0][0]]
    filtered_xfrom_user_dict[user] = cleaned_name_list

# Add inversions of lastname/firstname to all users
for user, name_list in filtered_xfrom_user_dict.items():
    new_name_set = set(name_list)
    for name in name_list:
        if ',' in name:
            # For all `lastname, firstname`, add `firstname lastname`
            lname, fname = name.split(',')
            new_name = fname.strip() + ' ' + lname.strip()
            new_name_set.add(new_name)
        else:
            # For all `firstname lastname`, add `lastname, firstname`
            names = name.split(' ')
            new_name = names[-1].strip() + ', ' + ' '.join(names[:-1]).strip()
            new_name_set.add(new_name)
    filtered_xfrom_user_dict[user] = list(new_name_set)

# Manual editions for bogus cases
filtered_xfrom_user_dict['hodge-j'] = ['hodge, john', 'hodge, jeffrey t.', 'hodge, jeffrey t', 'jeffrey t. hodge']
filtered_xfrom_user_dict['crandell-s'] = ['crandall, sean', 'sean crandall']

Make a dataframe to count the number of names by user, and manually check the users with the largest number of names to investigate potential bogus cases.

In [ ]:
counter = list(map(lambda e: (e[0], len(e[1])), filtered_xfrom_user_dict.items()))
df_xfrom_count = pd.DataFrame(counter, columns=['user_id', 'count'])
df_xfrom_count.sort_values(by='count').iloc[-5:]

#### Match user ids with X-To fields

In [ ]:
def get_user_inbox_mask(user):
    return (df.user_id == user) & (df.folder == 'inbox')

xto_user_dict = {}
for user in df.user_id.unique():
    user_inbox_mask = get_user_inbox_mask(user)
    if sum(user_inbox_mask) > 0:
        arr = np.hstack([list(map(clean_match, reg_xname.finditer(xto_str))) for xto_str in df[user_inbox_mask]['X-To'].values])
        xto_user_dict[user] = set(arr)
    else:
        print('Error on user', user)

Make a dataframe to count the number of names of a user shared both in sent and inbox messages, and check that no intersection is null.

In [ ]:
df_xto_count = pd.DataFrame([(user, set(from_name_list).intersection(set(xto_user_dict[user]))) for user, from_name_list in filtered_xfrom_user_dict.items() if user in xto_user_dict],
                            columns=['user_id', 'xfrom_xto_intersection'])
df_xto_count['count'] = df_xto_count.xfrom_xto_intersection.apply(len)

df_xto_count.sort_values(by='count', ascending=False).tail(5)

Build the mapping `X-To`/`X-From` name to `user_id`.

In [ ]:
xname_to_userid_mapping = {name: user for user, name_list in filtered_xfrom_user_dict.items() for name in name_list}

# Check that it has the same size of filtered_xfrom_user_dict (i.e. check that no names were duplicated and overriden in the dict)
assert sum(map(len, filtered_xfrom_user_dict.values())) == len(xname_to_userid_mapping)

In [ ]:
def extract_sender_id(xfrom_field):
    match = reg_xname.search(xfrom_field)
    if match:
        return xname_to_userid_mapping.get(clean_match(match))
    else:
        return None

def extract_receiver_id_list(xto_field):
    receiver_id_list = list()
    for match in reg_xname.finditer(xto_field):
        if match:
            receiver_id_list.append(xname_to_userid_mapping.get(clean_match(match)))
    return receiver_id_list

df['sender_id'] = df['X-From'].apply(lambda s: extract_sender_id(s))
df['receiver_id_list'] = df['X-To'].apply(lambda s: extract_receiver_id_list(s))

### Step 2.3: Build the timeseries dataframe

We now format the resulting dataframe into times series of sent emails per user.

In [ ]:
df_timeseries = df[~df.sender_id.isnull()][['sender_id', 'timestamp']].groupby('sender_id')['timestamp'].apply(list).to_frame()
df_timeseries = df_timeseries.rename(columns={'timestamp': 'timestamps'})

# Remove emails before January 1, 2000
min_timestamp = time.mktime(datetime(2000,1,1).timetuple())
df_timeseries['timestamps'] = df_timeseries['timestamps'].apply(np.array)
df_timeseries['timestamps'] = df_timeseries['timestamps'].apply(np.unique)
df_timeseries['timestamps'] = df_timeseries['timestamps'].apply(lambda events_m: events_m[events_m > min_timestamp])

# Add the count of events per user
df_timeseries['num_events_m'] = df_timeseries['timestamps'].apply(len)

# Filter out users with less than 10 events
df_timeseries = df_timeseries[df_timeseries.num_events_m > 10]

df_timeseries.num_events_m.sum() # Output: 74294

Save time series to json file.

In [ ]:
df_timeseries.to_json('enron_dataset_timeseries.json')